**Projet 2 : Reconnaissance de chiffres manuscrits (MNIST)**

**Nicolas Cambon et Justine Fiévé - Master 1 Sciences Cognitives (2019)**

Pour ce dernier projet, nous utilisons un réseau convolutionnel (CNN) permettant la classification d'images, et plus précisément d'images contenant un chiffre manuscrit (de 0 à 9, un chiffre par image). L'avantage du CNN est que son architecture permet de traiter des séries temporelles (ici, les nuances de gris de chaque image).

Le but est donc que ce réseau arrive à identifier les différents chiffres manuscrits, en faisant le moins d'erreur possible. Pour cela, il aura à disposition 60 000 images lors de la phase d'apprentissage puis 10 000 images lors de la phase de test. On cherche également à savoir si la perte de précision du détail de la nuance de gris sur les pixels influence grandement la capacité du réseau à prédire le bon chiffre.

Nous supposons alors que cette perte du détail de nuances de gris n'influence pas les capacités de réussite du réseau convolutionnel.

On commence donc par récupérer les données d'apprentissage et de test.

In [0]:
%%bash
rm train_image_file.txt train_label_file.txt test_image_file.txt test_label_file.txt mnist.tgz
wget http://pageperso.lif.univ-mrs.fr/~alexis.nasr/Ens/MASCO_Apprentissage_Automatique/mnist.tgz
tar xvfz mnist.tgz
ls data

On met ensuite en forme les données d'apprentissage. Pour cela, on transforme les listes de listes en tableau numpy 60000 x 28 x 28 x 1 pour les données d'apprentissage et 10000 x 28 x 28 x 1 pour les données de tests dans le but qu'elles puissent être prises en compte par Keras. 

Nous faisons également la mise en forme des labels d'apprentissage et de test en tableau numpy 60000 x 10 et 10000 x 10 respectivement. Pour chaque label, nous avons mis une liste de 10 éléments à la place d'un seul pour que Keras puisse comparer sa sortie, étant une liste de 10 éléments, à notre solution.

In [2]:
import numpy as np

def lecture_images(nomFichierImages):
    print("lecture des donnees d'entrée depuis le fichier : ", nomFichierImages) # la fonction lecture-image essaie d'ouvrir le fichier, renvoyant une erreur si ce dernier n'exsite pas
    try:
        fichier = open(nomFichierImages,"r")
    except IOError:
        print("le fichier",nomFichierLabels, "n'existe pas")
        return None
    fichier_entier = fichier.read() # on lit le fichier
    fichier.close()   
    files = fichier_entier.split(" ") # on découpe les données du fichier par leur espace
    for i in range (0,len(files)):
      if files[i] == "\n0": # pour toutes les données "\n0", on les remplace par 0
        files[i]="0"
    if files[-1] == "\n": # si le dernier élément est un \n, on l'enlève
      files.pop(-1)
    image_nuance=[]
    full_images=[]
    for i in range(1,len(files)+1): # pour tous les éléments de files 
      if i %(28*28) == 0 : # si le reste de l'élément par 784 est nul
        full_images.append(files[i-1]) # on ajoute à la liste full_images le 784 ème pixel de l'image
        image_nuance.append(full_images) # on introduit l'image dans la liste image_nuance créant une liste comportant la liste des pixels pour chaque image
        full_images=[] # on remet à 0 la liste full_images
      else:
        full_images.append(files[i-1]) # on ajoute les éléments de l'image dans la liste full_images
    k = len(image_nuance)
    image_nuance = np.array(image_nuance) # transformation des données en tableau
    image_nuance = image_nuance.reshape(k,28,28,1) # transformation en tableau k x 28 x 28 x 1
    return(image_nuance) # on renvoie le tableau image_nuance
      
    
    

def lecture_labels(nomFichierLabels):
    print("lecture des donnees de sortie depuis le fichier : ", nomFichierLabels)
    try:
        fichier = open(nomFichierLabels,"r")
    except IOError:
        print("le fichier",nomFichierLabels, "n'existe pas")
        return None
    fichier_lab = fichier.read()
    fichier.close()
    lab = fichier_lab.split("\n") # on sépare les données par rapport au \n
    if len(lab[-1]) == 0 : # si le dernier élément est vide
      lab.pop(-1) # on l'enlève
    l = len(lab)
    ly = []
    for i in range(len(lab)): # création d'une liste de 10 éléments pour chaque label
        v = [0] * 10
        v[int(lab[i])] = 1
        ly.append(v)
 
    lab = np.array(ly)
    lab=lab.reshape(l,10)
    return(lab)
    
x_train = lecture_images('./data/train_image_file.txt')
y_train = lecture_labels('./data/train_label_file.txt')

print('x shape = ', x_train.shape)
print('y shape = ', y_train.shape)

x_test = lecture_images('./data/test_image_file.txt')
y_test = lecture_labels('./data/test_label_file.txt')

print('x shape = ', x_test.shape)
print('y shape = ', y_test.shape)


lecture des donnees d'entrée depuis le fichier :  ./data/train_image_file.txt
lecture des donnees de sortie depuis le fichier :  ./data/train_label_file.txt
x shape =  (60000, 28, 28, 1)
y shape =  (60000, 10)
lecture des donnees d'entrée depuis le fichier :  ./data/test_image_file.txt
lecture des donnees de sortie depuis le fichier :  ./data/test_label_file.txt
x shape =  (10000, 28, 28, 1)
y shape =  (10000, 10)


Construction et apprentissage du modèle.

Ici, on créé le réseau convolutionnel avec ses différentes couches de pooling étant : 
          - Sequential
          - Conv2D
          - MaxPooling2D
          - Conv2D
          - MaxPooling2D
          - Dropout
          - Flatten
          - Dense

Le réseau reçoit en entrée (x) les différentes images de 28 x 28 pixels. Elles sont sous la forme de 784 séquences de nombres compris entre 0 (blanc) et 255 (noir) représentant les nuances de gris. Ces nuances de gris illustrent un chiffre.

Pour la sortie (y), le réseau reçoit la réponse et la compare avec sa prédiction (y'). Nous affichons pour l'apprentissage et les tests son taux de réussite en pourcentage ainsi que la focntion de perte.

In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D



batch_size = 128
num_classes = 10
epochs = 10
img_rows, img_cols = 28, 28

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=x_train.shape[1:]))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))
#model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,validation_data=(x_test, y_test))



Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 65s 1ms/step - loss: 1.6375 - acc: 0.8293 - val_loss: 0.1146 - val_acc: 0.9650
Epoch 2/10
60000/60000 [==============================] - 65s 1ms/step - loss: 0.1340 - acc: 0.9608 - val_loss: 0.0716 - val_acc: 0.9786
Epoch 3/10
60000/60000 [==============================] - 64s 1ms/step - loss: 0.0900 - acc: 0.9733 - val_loss: 0.0502 - val_acc: 0.9850
Epoch 4/10
60000/60000 [==============================] - 64s 1ms/step - loss: 0.0705 - acc: 0.9780 - val_loss: 0.0471 - val_acc: 0.9853
Epoch 5/10
60000/60000 [==============================] - 64s 1ms/step - loss: 0.0598 - acc: 0.9819 - val_loss: 0.0387 - val_acc: 0.9876
Epoch 6/10
60000/60000 [======

On peut observer que dès la première Epoch, le réseau a environ 83 % de réussite pour les données d'apprentissage et environ 97 % pour les données de tests. Au bout de 10 Epoch, il atteint, en moyenne, 98 % pour l'apprentissage et le test indiquant que le réseau a de bonnes performances prédictives. La fonction de perte, quant à elle, diminue.

Après l'avoir testé, nous essayons de voir si les séquences de nuances de gris sont importantes pour que le réseau réponde correctement. Nous avons donc créer une fonction lecture_images_col qui transforme les images "en noir et blanc". En d'autres termes, tous les chiffres différents de 0 sont changés en 1 (les 0 restent tels quels), il n'y a plus de nuances de gris. 

In [4]:
def lecture_images_col(nomFichierImages):
    print("lecture des donnees d'entrée depuis le fichier : ", nomFichierImages)
    try:
        fichier = open(nomFichierImages,"r")
    except IOError:
        print("le fichier",nomFichierLabels, "n'existe pas")
        return None
    fichier_entier = fichier.read()
    fichier.close()  
    files = fichier_entier.split(" ")
    for i in range (0,len(files)):
      if files[i] == "\n0":
        files[i]="0"
    if files[-1] == "\n":
      files.pop(-1)
    image_nuance=[]
    full_images=[]
    for i in range(1,len(files)+1):
      if i %(28*28) == 0 :
        if int(files[i-1]) == 0: # si l'élement est égale à 0, ajout de 0 dans full_images
          full_images.append(0)
          image_nuance.append(full_images)
          full_images=[]
        else:  # sinon, l'élement est égale à 1
          full_images.append(1)
          image_nuance.append(full_images)
          full_images=[]
      else:
        if int(files[i-1]) == 0:
          full_images.append(0)
        else :
          full_images.append(1)
    k = len(image_nuance)
    image_nuance = np.array(image_nuance)
    image_nuance = image_nuance.reshape(k,28,28,1)
    return(image_nuance)
  
x_train_col = lecture_images_col('./data/train_image_file.txt') 
print('x shape = ', x_train_col.shape)

x_test_col = lecture_images_col('./data/test_image_file.txt')
print('x shape = ', x_test_col.shape)



lecture des donnees d'entrée depuis le fichier :  ./data/train_image_file.txt
x shape =  (60000, 28, 28, 1)
lecture des donnees d'entrée depuis le fichier :  ./data/test_image_file.txt
x shape =  (10000, 28, 28, 1)


In [5]:
model.fit(x_train_col, y_train,batch_size=batch_size,epochs=epochs,validation_data=(x_test_col, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 46s 761us/step - loss: 0.5019 - acc: 0.8415 - val_loss: 0.1163 - val_acc: 0.9668
Epoch 2/10
60000/60000 [==============================] - 45s 753us/step - loss: 0.1296 - acc: 0.9603 - val_loss: 0.0755 - val_acc: 0.9766
Epoch 3/10
60000/60000 [==============================] - 45s 758us/step - loss: 0.1023 - acc: 0.9681 - val_loss: 0.0646 - val_acc: 0.9803
Epoch 4/10
60000/60000 [==============================] - 46s 759us/step - loss: 0.0902 - acc: 0.9718 - val_loss: 0.0611 - val_acc: 0.9809
Epoch 5/10
60000/60000 [==============================] - 45s 755us/step - loss: 0.0818 - acc: 0.9747 - val_loss: 0.0558 - val_acc: 0.9828
Epoch 6/10
60000/60000 [==============================] - 46s 759us/step - loss: 0.0756 - acc: 0.9761 - val_loss: 0.0533 - val_acc: 0.9837
Epoch 7/10
60000/60000 [==============================] - 46s 759us/step - loss: 0.0714 - acc: 0.9784 - val_loss: 0.

Également, on peut observer que le réseau arrive à environ 98 % de réussite pour les données d'apprentissage et de tests au bout de 10 Epoch. La fonction de perte diminue au fur et à mesure des tests. On peut donc penser que les nuances de gris n'influencent pas l'apprentissage du réseau convolutionnel, comme nous l'avons supposé en introduction.

Par la suite, nous avons voulu tester si le réseau était sensible à la translation ou à la rotation des images. Nous avons donc, pour commencer, essayer la translation des images.

In [6]:
def lecture_images_translation(nomFichierImages):
    print("lecture des donnees d'entrée depuis le fichier : ", nomFichierImages)
    try:
        fichier = open(nomFichierImages,"r")
    except IOError:
        print("le fichier",nomFichierLabels, "n'existe pas")
        return None
    fichier_entier = fichier.read()
    fichier.close()  
    files = fichier_entier.split(" ")
    for i in range (0,len(files)):
      if files[i] == "\n0":
        files[i]="0"
    if files[-1] == "\n":
      files.pop(-1)
    image_nuance=[]
    full_images=[]
    for i in range(1,len(files)+1):
      if i %(28*28) == 0 : 
        full_images.insert(0,files[i-1]) # pour chaque ligne de pixels, on inverse leur position
        image_nuance.append(full_images)
        full_images=[]
      else:
        full_images.insert(0,files[i-1])
    k = len(image_nuance)
    image_nuance = np.array(image_nuance)
    image_nuance = image_nuance.reshape(k,28,28,1)
    return(image_nuance)
  
x_train_trans = lecture_images_translation('./data/train_image_file.txt') 
print('x shape = ', x_train_trans.shape)

x_test_trans = lecture_images_translation('./data/test_image_file.txt')
print('x shape = ', x_test_trans.shape)


lecture des donnees d'entrée depuis le fichier :  ./data/train_image_file.txt
x shape =  (60000, 28, 28, 1)
lecture des donnees d'entrée depuis le fichier :  ./data/test_image_file.txt
x shape =  (10000, 28, 28, 1)


In [7]:
model.fit(x_train_trans, y_train,batch_size=batch_size,epochs=epochs,validation_data=(x_test_trans, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 64s 1ms/step - loss: 8.8732 - acc: 0.4485 - val_loss: 8.0352 - val_acc: 0.5011
Epoch 2/10
60000/60000 [==============================] - 64s 1ms/step - loss: 8.1141 - acc: 0.4958 - val_loss: 7.5280 - val_acc: 0.5324
Epoch 3/10
60000/60000 [==============================] - 65s 1ms/step - loss: 7.5708 - acc: 0.5295 - val_loss: 6.8939 - val_acc: 0.5716
Epoch 4/10
60000/60000 [==============================] - 64s 1ms/step - loss: 7.1596 - acc: 0.5552 - val_loss: 6.7987 - val_acc: 0.5777
Epoch 5/10
60000/60000 [==============================] - 64s 1ms/step - loss: 7.0024 - acc: 0.5649 - val_loss: 6.7193 - val_acc: 0.5829
Epoch 6/10
60000/60000 [==============================] - 64s 1ms/step - loss: 6.9332 - acc: 0.5694 - val_loss: 6.6421 - val_acc: 0.5878
Epoch 7/10
60000/60000 [==============================] - 64s 1ms/step - loss: 6.8787 - acc: 0.5727 - val_loss: 6.6564 - val_acc

On observe que dans ce cas-là, le réseau a un taux de réussite d'environ 60 % pour les données d'apprentissage et de tests et que la fonction de perte est plus élevée comparés aux tests précédents. La translation impacte les capacités de prédiction du réseau.

Nous testons ensuite la rotation des images.

In [8]:
def lecture_images_inversion(nomFichierImages):
    print("lecture des donnees d'entrée depuis le fichier : ", nomFichierImages)
    try:
        fichier = open(nomFichierImages,"r")
    except IOError:
        print("le fichier",nomFichierLabels, "n'existe pas")
        return None
    fichier_entier = fichier.read()
    fichier.close()  
    files = fichier_entier.split(" ")
    for i in range (0,len(files)):
      if files[i] == "\n0":
        files[i]="0"
    if files[-1] == "\n":
      files.pop(-1)
    image_nuance=[]
    full_images=[]
    for i in range(1,len(files)+1):
      if i %(28*28) == 0 :
        full_images.append(files[i-1])
        image_nuance.insert(0,full_images) # pour chaque colonne de pixels, on inverse leur position
        full_images=[]
      else:
        full_images.append(files[i-1])
    k = len(image_nuance)
    image_nuance = np.array(image_nuance)
    image_nuance = image_nuance.reshape(k,28,28,1)
    return(image_nuance)
  
x_train_inv = lecture_images_inversion('./data/train_image_file.txt') 
print('x shape = ', x_train_inv.shape)

x_test_inv = lecture_images_inversion('./data/test_image_file.txt')
print('x shape = ', x_test_inv.shape)


lecture des donnees d'entrée depuis le fichier :  ./data/train_image_file.txt
x shape =  (60000, 28, 28, 1)
lecture des donnees d'entrée depuis le fichier :  ./data/test_image_file.txt
x shape =  (10000, 28, 28, 1)


In [9]:
model.fit(x_train_inv, y_train,batch_size=batch_size,epochs=epochs,validation_data=(x_test_inv, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 64s 1ms/step - loss: 14.5336 - acc: 0.0975 - val_loss: 14.5069 - val_acc: 0.0994
Epoch 2/10
60000/60000 [==============================] - 64s 1ms/step - loss: 14.5397 - acc: 0.0972 - val_loss: 14.5530 - val_acc: 0.0968
Epoch 3/10
60000/60000 [==============================] - 64s 1ms/step - loss: 14.5422 - acc: 0.0972 - val_loss: 14.5701 - val_acc: 0.0958
Epoch 4/10
60000/60000 [==============================] - 64s 1ms/step - loss: 14.5254 - acc: 0.0983 - val_loss: 14.5130 - val_acc: 0.0987
Epoch 5/10
60000/60000 [==============================] - 64s 1ms/step - loss: 14.5340 - acc: 0.0978 - val_loss: 14.5506 - val_acc: 0.0969
Epoch 6/10
60000/60000 [==============================] - 64s 1ms/step - loss: 14.5461 - acc: 0.0974 - val_loss: 14.5239 - val_acc: 0.0988
Epoch 7/10
60000/60000 [==============================] - 64s 1ms/step - loss: 14.5395 - acc: 0.0979 - val_loss: 14.

Dans le cas de la rotation, le réseau n'arrive pas du tout à identifer les chiffres manuscrits. En effet, il a un taux de réussite de seulement 10 % pour les données d'apprentissage et de tests, on peut dire qu'il répond aléatoirement. Ainsi, la rotation des données annihile complètement les capacités de prédiction du réseau.

**Conclusion :**

**D'après ces observations, nous pouvons dire que lorsque nous modifions seulement des détails dans les données (ici, les nuances de gris), le réseau convolutionnel garde intacte ses capacités de prédiction (environ 98 % de réussite pour les données d'apprentissage et de tests).**

**Cependant, lorsque nous changeons leur nature, les données deviennent plus difficiles à être traitées par le réseau, diminuant alors grandement ses capacités de prédiction (60 % de réussite pour la translation et 10 % de réussite pour la rotation pour les données d'apprentissage et de test). Le réseau n'est donc pas adapté pour traiter cette configuration.**